In [22]:
import requests
import json
import os
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML
import base64
from PIL import Image as PILImage, ImageDraw
import numpy as np

In [ ]:
#Lecture du fichier de données des joueurs
player_file_path = os.path.join('..', 'data', 'nhl_player_data.json')
if os.path.exists(player_file_path):
        with open(player_file_path, 'r') as file:
            try:
                # Load the JSON file into the dictionary
                player_names = json.load(file)
                print(f"Loaded {len(player_names)} players from the file.")
            except json.JSONDecodeError as e:
                print(f"Error reading player data: {e}")
else:
    print(f"Player file {player_file_path} not found.")
    player_names = {}

def get_player_name(player_id: int) -> str:
    """
    Retrieves the name of a player given their ID.

    Args:
        player_id (int): The ID of the player to retrieve.
    
    Returns:
        str: The name of the player or 'Unknown Player' if not found.
    """
    if player_id is None:
        return None

    # Vérifier si le joueur est déjà dans le dictionnaire
    if str(player_id) in player_names:
        return player_names[str(player_id)]
    
    # Effectuer une requête à l'API pour récupérer les informations du joueur
    url = f"https://api-web.nhle.com/v1/player/{player_id}/landing"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lève une erreur si la requête échoue
        player_data = response.json()
        
        # Extraire les noms du joueur (en accédant à la clé 'default')
        first_name = player_data.get('firstName', {}).get('default', 'Unknown')
        last_name = player_data.get('lastName', {}).get('default', 'Player')
        full_name = f"{first_name} {last_name}"

        # Stocker dans le dictionnaire pour éviter des appels répétés
        player_names[str(player_id)] = full_name
        
        return full_name

    except requests.RequestException as e:
        print(f"Error fetching player data: {e}")
        return "Unknown Player"

 


def describe_event(event):
    # Extraire les informations de l'événement
    typeDescKey = event.get('typeDescKey', "")
    details = event.get('details', {})

    # Récupérer les IDs des joueurs s'ils sont présents
    player_ids = {
        "losing": details.get('losingPlayerId'),
        "winning": details.get('winningPlayerId'),
        "giveaway": details.get('playerId'),
        "blocking": details.get('blockingPlayerId'),
        "blocked": details.get('shootingPlayerId'),
        "shooting": details.get('shootingPlayerId'),
        "goalie": details.get('goalieInNetId'),
        "penalized": details.get('committedByPlayerId'),
        "penalty": details.get('drawnByPlayerId'),
        "hitting": details.get('hittingPlayerId'),
        "hit": details.get('hitteePlayerId')
    }

    # Récupérer les noms des joueurs si les IDs sont présents
    players = {key: get_player_name(player_id) for key, player_id in player_ids.items() if player_id}

    # Créer une description basée sur le type d'événement
    event_descriptions = {
        "faceoff": lambda: f"{players['winning']} won a faceoff against {players['losing']}.",
        "goal": lambda: f"{players['shooting']} scored a goal against {players['goalie']}.",
        "shot-on-goal": lambda: f"{players['shooting']} took a shot on goal saved by {players['goalie']}.",
        "giveaway": lambda: f"{players['giveaway']} gave the puck away.",
        "period-start": lambda: "The period has started.",
        "blocked-shot": lambda: f"{players['blocking']} blocked a shot from {players['blocked']}.",
        "delayed-penalty": lambda: "A delayed penalty was called.",
        "failed-shot-attempt": lambda: f"{players['shooting']} attempted a shot but missed the net.",
        "shootout-complete": lambda: "The shootout has ended.",
        "stoppage": lambda: "The game was stopped.",
        "penalty": lambda: f"Penalty: {players['penalized']} committed a foul against {players['penalty']}.",
        "takeaway": lambda: f"{players['giveaway']} took the puck away.",
        "missed-shot": lambda: f"{players['shooting']} missed the net with a shot.",
        "game-end": lambda: "The game has ended.",
        "hit": lambda: f"{players['hitting']} delivered a hit on {players['hit']}.",
        "period-end": lambda: "The period has ended."
    }

    # Retourner la description ou un message par défaut
    return event_descriptions.get(typeDescKey, lambda: "Event description not available.")()





In [24]:
def normalize_coordinates(x, y, img_width, img_height):
    """
    Convertit les coordonnées NHL standard (x, y) en coordonnées normalisées sur une image de patinoire.
    Les dimensions de la patinoire sont automatiquement adaptées à la taille totale de l'image.
    
    Arguments:
    x, y : Coordonnées NHL standard (x: -100 à 100, y: -42.5 à 42.5)
    img_width : Largeur totale de l'image
    img_height : Hauteur totale de l'image
    
    Retourne:
    (x_img, y_img) : Coordonnées normalisées pour l'image.
    """
    # Normaliser les coordonnées (x: -100 à 100) et (y: -42.5 à 42.5)
    x_norm = (x + 100) / 200  # Passer de l'échelle (-100, 100) à (0, 1)
    y_norm = (42.5 - y) / 85  # Passer de l'échelle (-42.5, 42.5) à (0, 1)

    # Convertir les coordonnées normalisées en pixels sur l'image complète
    x_img = img_width * x_norm
    y_img = img_height * y_norm

    return x_img, y_img


In [25]:
def draw_grid_and_labels(draw, img_width, img_height):
    """
    Dessine des repères sur l'image de la patinoire.
    
    Arguments:
    draw : objet ImageDraw pour dessiner sur l'image
    img_width : largeur de l'image
    img_height : hauteur de l'image
    """
    # Définir les positions des repères
    x_ticks = [-100, -75, -50, -25, 0, 25, 50, 75, 100]
    y_ticks = [-42.5, -21.25, 0, 21.25, 42.5]
    
    # Couleurs et taille de police
    line_color = "gray"
    font_color = "black"
    
    # Dessiner les lignes de repère pour x
    for x in x_ticks:
        x_img = (x + 100) / 200 * img_width  # Normaliser x
       
        draw.text((x_img + 5, img_height - 20), str(x), fill=font_color)  # Étiquettes x

    # Dessiner les lignes de repère pour y
    for y in y_ticks:
        y_img = (42.5 - y) / 85 * img_height  # Normaliser y
        
        draw.text((5, y_img - 10), str(y), fill=font_color)  # Étiquettes y


In [26]:
def playerId_type(event):
    event_type = event.get('typeDescKey', 'N/A')
    details = event.get('details', {})
    
    if event_type == 'hit':
        return {
            "player_1": {"full name": get_player_name(details.get('hittingPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('hitteePlayerId'))}
        }
    elif event_type == 'faceoff':
        return {
            "player_1": {"full name": get_player_name(details.get('winningPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('losingPlayerId'))}
        }
    elif event_type == 'blocked-shot':
        return {
            "player_1": {"full name": get_player_name(details.get('blockingPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('shootingPlayerId'))}
        }
    elif event_type in ['missed-shot', 'shot-on-goal']:
        return {
            "player_1": {"full name": get_player_name(details.get('shootingPlayerId'))},
            "player_2": {"full name": get_player_name(details.get('goalieInNetId'))}
        }
    elif event_type == 'giveaway':
        return {
            "player_1": {"full name": get_player_name(details.get('playerId'))}
        }
    else:
        return {}

In [ ]:
class NHLDataParser:
    def __init__(self, nhl_games_file_path, season=None, stage='regular'):
        self.season = season
        self.stage = stage
        self.game_type = {'regular': 2, 'playoff': 3}

        if os.path.isdir(nhl_games_file_path):
            json_file_path = os.path.join(nhl_games_file_path, 'all_games_data.json')
        else:
            json_file_path = nhl_games_file_path

        with open(json_file_path, 'r') as file:
            self.games_data = json.load(file)

        self.filtered_games = self.filter_games_by_season_and_stage()

    def filter_games_by_season_and_stage(self):
        filtered = {}
        for game_id, game_data in self.games_data.items():
            if (self.season is None or game_data['season'] == int(self.season)) and game_data['gameType'] == self.game_type[self.stage]:
                filtered[game_id] = game_data
        return filtered

    def get_game_ids(self):
        return list(self.filtered_games.keys())

    def get_events(self, game_id):
        return self.filtered_games[game_id]['plays'] if game_id in self.filtered_games else []
    
    

    def get_game_info(self, game_id):
        """Retourne les informations du match (Game ID, équipes, score, etc.)."""
        game_data = self.filtered_games.get(game_id)
        if not game_data:
            return "Données du match indisponibles"
        
        home_team = game_data['homeTeam']['name']['default']
        away_team = game_data['awayTeam']['name']['default']
        home_score = game_data['homeTeam']['score']
        away_score = game_data['awayTeam']['score']
        game_starting_time = game_data['startTimeUTC']
        home_sog = game_data['homeTeam']['sog']
        away_sog = game_data['awayTeam']['sog']

        formatted_info = f"""
        <p><b>{game_starting_time}</b></p>
        <p><b>Game ID: {game_id}; {home_team} (home) vs {away_team} (away)</b></p>
        <p><b>{game_data['periodDescriptor']['periodType']}</b></p>
        <table style="width:50%; text-align: left; border-collapse: collapse;">
           <tr>
               <th></th>
               <th style="text-align: center;">Home</th>
               <th style="text-align: center;">Away</th>
           </tr>
           <tr>
               <td><b>Teams:</b></td>
               <td style="text-align: center;">{home_team}</td>
               <td style="text-align: center;">{away_team}</td>
           </tr>
           <tr>
               <td><b>Goals:</b></td>
               <td style="text-align: center;">{home_score}</td>
               <td style="text-align: center;">{away_score}</td>
           </tr>
           <tr>
               <td><b>SoG:</b></td>
               <td style="text-align: center;">{home_sog}</td>
               <td style="text-align: center;">{away_sog}</td>
           </tr>
       </table>
       """
        return formatted_info

    def get_event_coordinates(self, event):
        """Retourne les coordonnées x et y d'un événement."""
        return event.get('details', {}).get('xCoord'), event.get('details', {}).get('yCoord')


# Fonction pour afficher la patinoire avec la position du joueur marquée
def show_rink_with_player_position(x, y, rink_display_label,event):
    # Charger l'image de la patinoire
    root_directory = os.path.abspath(os.path.join('..'))
    img_path = os.path.join(root_directory, 'figures', 'nhl_rink.png')
    

    if x is not None and y is not None:
        # Ouvrir l'image
        img = PILImage.open(img_path)

        if img.mode != 'RGB':
           img = img.convert('RGB')
        draw = ImageDraw.Draw(img)

         # Obtenir les dimensions de l'image complète
        img_width, img_height = img.size

        # Dessiner les repères
        draw_grid_and_labels(draw, img_width, img_height)

        # Normaliser les coordonnées du joueur par rapport à la taille de l'image complète
        x_img, y_img = normalize_coordinates(x, y, img_width, img_height)

        # Dessiner un point ou une forme correspondant à l'événement
        point_size = 10
        if event.get('typeDescKey', 'N/A') == 'missed-shot':
            draw.ellipse([(x_img - point_size, y_img - point_size), 
                          (x_img + point_size, y_img + point_size)], outline='red', width=2)
            draw.line([(x_img - point_size, y_img - point_size), 
                       (x_img + point_size, y_img + point_size)], fill='red', width=2)
        else:
            draw.ellipse([(x_img - point_size, y_img - point_size), 
                          (x_img + point_size, y_img + point_size)], fill='blue')

        # Sauvegarder et afficher l'image temporairement
        temp_image_path = os.path.join(root_directory, 'figures', 'temp_rink_image.png')
        img.save(temp_image_path)

        # Afficher l'image en base64
        with open(temp_image_path, "rb") as img_file:
            b64_string = base64.b64encode(img_file.read()).decode('utf-8')

        rink_display_label.clear_output()
        with rink_display_label:
            display(HTML(f'<img src="data:image/png;base64,{b64_string}" style="width: 50%; height: auto;" />'))
    else:
        rink_display_label.clear_output()
        


# Initialisation du parser
root_directory = os.path.abspath(os.path.join('..'))
nhl_games_file_path = os.path.join(root_directory,'data', 'nhl_game_data.json')
parser = NHLDataParser(nhl_games_file_path, season='20162017', stage='regular')

# Widgets pour l'interface utilisateur
game_ids = parser.get_game_ids()
game_id_slider = widgets.IntSlider(
    value=int(game_ids[0]),
    min=int(game_ids[0]),
    max=int(game_ids[-1]),
    step=1,
    description='Game ID:',
    continuous_update=False
)

event_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=0,
    step=1,
    description='Event:',
    continuous_update=False
)

# Créer un ButtonSlider
button_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Sélectionnez une valeur:',
    continuous_update=False
)

game_info_label = widgets.Output()
event_info_label = widgets.Output()
rink_display_label = widgets.Output()
even_description_label = widgets.Output(layout=widgets.Layout(display='flex', margin = '0 100px'))



# Fonction pour mettre à jour les événements en fonction du `game_id` sélectionné
def update_event_slider(*args):
    game_id = str(game_id_slider.value)
    events = parser.get_events(game_id)
    event_slider.max = len(events) - 1 if events else 0

    # Mettre à jour les informations du match
    with game_info_label:
        game_info_label.clear_output()
        display(HTML(parser.get_game_info(game_id)))

    # Mettre à jour le slider des événements
    event_slider.value = 0  # Réinitialiser à 0 chaque fois qu'on change de match


def format_dict(d, indent=0):
    result = ""
    for key, value in d.items():
        # Vérifier si la clé est l'une des sections spéciales
        if key in ['About', 'Results', 'Players']:
            result += '  ' * indent + f"<b>{key}:</b> "  # Ajoute les balises <b> autour du titre
        else:
            result += '  ' * indent + f"{key}: "
        
        if isinstance(value, dict):
            result += "\n{\n" + format_dict(value, indent + 1) + '  ' * indent + "}\n"
        else:
            result += f"{value}\n"
    return result


# Fonction pour afficher l'événement sélectionné
def display_event(game_id, event_idx):
    clear_output(wait=True)
    display(vbox)

    events = parser.get_events(str(game_id))
    if events and 0 <= event_idx < len(events):
        event = events[event_idx]
        
        # Récupérer la description de l'événement
        event_description = describe_event(event)

    
        
        # Afficher les informations de l'événement
        # Créer le dictionnaire des informations de l'événement
        event_info = {
             "About": {
        "eventId": event.get('eventId', 'N/A'),
        "periodType": event.get('periodDescriptor', {}).get('periodType', 'N/A'),
        "maxRegulationPeriods": event.get('periodDescriptor', {}).get('maxRegulationPeriods', 'N/A'),
        
        "timeInPeriod": event.get('timeInPeriod', 'N/A'),
        "timeRemaining": event.get('timeRemaining', 'N/A'),
        "situationCode": event.get('situationCode', 'N/A'),
        "homeTeamDefendingSide": event.get('homeTeamDefendingSide', 'N/A'),
        "typeCode": event.get('typeCode', 'N/A'),
        
        "sortOrder": event.get('sortOrder', 'N/A'),
        "coordinates": {"x": event.get('details', {}).get('xCoord', 'N/A'),"y": event.get('details', {}).get('yCoord', 'N/A') }
           },
          "Players":{
        "Players": playerId_type(event)
             },
        "Results":{"description":describe_event(event),
                   "event": event.get('typeDescKey', 'N/A'),
                   }

        }
        


        formatted_info = format_dict(event_info)
        
        # Afficher l'image de la patinoire avec les coordonnées de l'événement
        x_coord, y_coord = parser.get_event_coordinates(event)

        # Afficher l'image de la patinoire avec le joueur et le titre comme description de l'événement
        show_rink_with_player_position(x_coord, y_coord, rink_display_label,event)


        # Mettre à jour le even_description_label avec la description de l'événement
        with even_description_label:
            even_description_label.clear_output()
            display(HTML(f'<h4>{event_description}</h4>'))  # Affichage de la description
        
        # Afficher la description de l'événement comme titre
        with event_info_label:
            event_info_label.clear_output()
            display(HTML(f'<pre style="font-size:10px;">{formatted_info}</pre>'))

    else:
        with event_info_label:
            event_info_label.clear_output()
            display("No event data available for this game or invalid event index.")



# Associer les changements de sliders aux événements
game_id_slider.observe(update_event_slider, 'value')
event_slider.observe(lambda change: display_event(game_id_slider.value, event_slider.value), 'value')




# Conteneur pour afficher les widgets
vbox = widgets.VBox([
    game_id_slider,
    game_info_label,
    event_slider,
    even_description_label,
    rink_display_label,
    event_info_label
    
])
display(vbox)



# Initialiser l'affichage
update_event_slider()